# Deepseek LLM Grader

## Lib import

In [ ]:
import os
import logging
from typing import List, Literal, Annotated, Optional, Union
from typing_extensions import TypedDict
import chromadb
from chromadb.config import Settings
from pydantic import BaseModel, Field, validator

from langchain import PromptTemplate, LLMChain
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnableSerializable
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_chroma import Chroma
from langchain_text_splitters import TokenTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.schema import Document
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_huggingface import HuggingFaceEmbeddings

from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import InMemorySaver

## Init and API key

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# key
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
silicon_api_key = os.getenv("SILICON_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# deepseek
deepseek_llm_model = "deepseek-chat"

# silicon
silicon_base_url =  "https://api.siliconflow.cn/v1"
silicon_llm_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# huggingface
huggingface_embed_model = "sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
logging.basicConfig(level=logging.INFO)

## LLM Init

In [ ]:
# init LLM mod
llm_deepseek = ChatDeepSeek(
    model=deepseek_llm_model,
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    top_p=0.9,
    frequency_penalty=0.7,
    presence_penalty=0.5,
    max_retries=3
)

## Grader

In [ ]:
################################################################################
###  Grader
################################################################################
class Grader:
    """
    Grader.
    The score to assess relevance or answer addresses question.
    """

    def __init__(
        self,
        llm: RunnableSerializable,
        grader_type: Literal["relevance", "answer"],
    ):
        self.grader_type = grader_type
        self.schema = self._default_schema()
        self.system_prompt = self._default_system_prompt()
        self.chain = self._build_chain(llm)

    @staticmethod
    def _default_schema():
        score_min = 0
        score_max = 10
        class GradeSchema(BaseModel):
            score: int = Field(
                default=0,
                description=(
                    """
                    \rScore between 0-10
                    """
                ),
                examples=[3, 7, 9],
                ge=score_min,
                le=score_max
            )
            
            def cast_to_int(cls, value):
                return int(float(value))

            def validate_range(cls, value):
                if value < score_min:
                    value = score_min
                elif value > score_max:
                    value = score_max;
                    
                return value

        return GradeSchema
    
    #@staticmethod
    def _default_system_prompt(self):
        instructs = {
            "relevance": "relevance of the document to the question",
            "answer": "answer addresses the question",
        }
        instruct = instructs[self.grader_type]
        
        role = f"""
            \rRole: Scoring Expert
            \rTask: {instruct}
            \r
            \rGuidelines:
            \rYou are a professional scoring expert. Please assess the {instruct} based on the following criteria:
            \rScoring Criteria:
            \r0: Irrelevant/no answer
            \r3: Only keywords/no relevance
            \r5: Relevant keywords, partial answer
            \r7: Mostly relevant, insufficient support
            \r9: Highly relevant, detailed evidence
            \r10: Perfect solution/answer
        """

        respond_request = """
            \rEvaluation Criteria: Keyword Match, Relevance, Evidence Strength
            \rCurrent Scenario: Rapid Response Mode
        """

        response_format = """
            \rPlease return a number score from 0-10.
            \rNo explanations or additional text.
        """
    
        return f"{role}\n{respond_request}\n{response_format}"
    
    def _build_chain(self, llm: RunnableSerializable) -> RunnableSerializable:
        structured_llm = llm.with_structured_output(self.schema)

        instructs = {
            "relevance": "Rrelevance document",
            "answer": "Aanswer",
        }
        instruct = instructs[self.grader_type]
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            ("human", f"{instruct}: \n\n {{document}} \n\n Question: {{question}}")
        ])
        
        return prompt_template | structured_llm
    
    def invoke(
        self,
        document: str,
        question: str,
        **kwargs
    ) -> Union[str, dict]:
        inputs = {
            "document": document,
            "question": question,
            **kwargs
        }
        return self.chain.invoke(inputs)

## Test Grader

### Test relevance grader

In [ ]:
relevance_grader = Grader(llm=llm_deepseek, grader_type="relevance")
print("=== relevance grader system_prompt ===")
print(relevance_grader.system_prompt)

In [ ]:
##
## Test Grade: yes
##

user_input = "What are the ingredients in Alpha Hope?"
retrieval_document = """
Alpha Hope has been formulated with two powerful active ingredients, PQQ and Molecular Hydrogen.
They work synergistically to activate metabolic pathways involved in energy production and cognition.
This is particularly formulated to promote the bodyâ\x80\x99s natural detox process and help the body naturally produce Hope Molecules,\
also known as PGC-1Î±, that fight oxidative damage.
"""
grade = relevance_grader.invoke(question=user_input, document=retrieval_document)
print(f"relevance grade is {grade.score}")

### Test answer grader

In [ ]:
answer_grader = Grader(llm=llm_deepseek, grader_type="answer")
print("=== answer grader system_prompt ===")
print(answer_grader.system_prompt)

In [ ]:
##
## Test Grade: no
##

user_input = "What are the ingredients in Alpha Hope?"
retrieval_document = """
Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager, possibly rendering your system unusable.
It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action option if you know what you are doing and want to suppress this warning.
"""

grade = answer_grader.invoke(question=user_input, document=retrieval_document)
print(f"answer grade is {grade.score}")